In [25]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from sklearn.preprocessing import MinMaxScaler
from IPython.display import display
colors = ['#082040', '#175073', '#3285A6', '#B8D0D9', '#6CC5D9']

In [26]:
# Load data and prepare it
df = pd.read_csv(r'/content/Megy_final.csv', index_col=0)

In [27]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 2143 entries, 0 to 2142
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 2143 non-null   int64  
 1   space_group_core   2143 non-null   int64  
 2   space_group_shell  2143 non-null   int64  
 3   Tc                 2143 non-null   int64  
 4   x                  2143 non-null   float64
 5   y                  2143 non-null   float64
 6   z                  2143 non-null   float64
 7   shape              2143 non-null   int64  
 8   temperature_k      2143 non-null   float64
 9   h_range_max_koe    2143 non-null   float64
 10  sat_em_g           2143 non-null   float64
 11  coer_oe            2143 non-null   float64
 12  mr (emu/g)         2143 non-null   float64
 13  sphericity         2143 non-null   float64
 14  max/min            2143 non-null   float64
 15  area/volume        2143 non-null   float64
 16  num_of_magn_ions   2143 non-n

In [28]:
removed_columns = ['id', 'link', 'orig_c1', 'orig_c2']
removed_columns_df = df[removed_columns]
df_for_cleaning = df.drop(columns=removed_columns) # Now df_for_cleaning contains all columns except the removed ones

In [29]:
# Select columns to clean
columns_to_clean = ['coer_oe', 'sat_em_g']

# Scale the entire dataset (except removed columns)
scaler = MinMaxScaler()
scaled_df = scaler.fit_transform(df_for_cleaning) # Scale all columns
df_scaled = pd.DataFrame(scaled_df, columns=df_for_cleaning.columns, index=df_for_cleaning.index)
print("Data scaled")

Data scaled


In [30]:
# Function to detect outliers (IQR)
def detect_outliers_iqr(data, threshold=1.5):
    quartile_1, quartile_3 = np.percentile(data, [20, 80])
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (threshold * iqr)
    upper_bound = quartile_3 + (threshold * iqr)
    outliers = np.where((data < lower_bound) | (data > upper_bound))[0]
    return outliers

In [31]:
# Detect outliers for the two columns (in scaled data)
outliers_coer_oe = detect_outliers_iqr(df_scaled['coer_oe'])
outliers_sat_em_g = detect_outliers_iqr(df_scaled['sat_em_g'])

# Combine outlier indices (unique values)
outliers = np.union1d(outliers_coer_oe, outliers_sat_em_g)  # All outliers from both columns

print("Outlier indices:", outliers)

Outlier indices: [   0   36   52   53   54   90   91   92  123  124  147  152  153  154
  155  156  157  190  191  253  254  257  360  361  362  374  375  376
  377  378  396  402  403  420  421  442  443  446  447  448  449  450
  469  488  489  490  491  492  496  502  507  508  512  513  939  940
  941  948  949  979 1083 1089 1091 1093 1095 1097 1099 1111 1112 1113
 1114 1115 1116 1118 1146 1147 1152 1155 1156 1157 1159 1161 1163 1164
 1165 1174 1175 1176 1180 1182 1183 1184 1185 1186 1187 1188 1191 1194
 1195 1227 1231 1246 1247 1249 1261 1279 1280 1281 1282 1285 1286 1287
 1288 1289 1290 1291 1292 1293 1294 1295 1356 1357 1358 1361 1362 1363
 1364 1365 1366 1367 1384 1387 1388 1394 1395 1396 1397 1398 1399 1416
 1417 1423 1441 1442 1443 1451 1452 1453 1454 1458 1459 1460 1461 1465
 1466 1467 1468 1469 1470 1478 1479 1500 1570 1571 1599 1600 1604 1631
 1637 1638 1639 1643 1644 1686 1687 1688 1689 1690 1691 1692 1693 1694
 1695 1696 1731 1733 1734 1735 1738 1739 1748 1749 1750 1751

In [32]:
# Visualization
outlier_values_coer_oe = df_for_cleaning['coer_oe'].iloc[outliers] # Use the original dataframe to get outlier values

fig = go.Figure()
fig.add_trace(go.Scatter(x=df_for_cleaning.index, y=df_for_cleaning['coer_oe'], mode='markers', name='Data Points', marker=dict(color=colors[2])))
fig.add_trace(go.Scatter(x=df_for_cleaning.index[outliers], y=outlier_values_coer_oe, mode='markers', marker=dict(color=colors[0]), name='Outliers'))
fig.update_layout(title='Outlier Detection (coer_oe)', xaxis_title='Index', yaxis_title='Value')
display(fig)

In [33]:
# Remove outliers from scaled data (from all columns, not just the cleaned ones)
out_data = df_scaled.drop(index=df_scaled.index[outliers])  # Remove rows with outliers from the scaled data

# Inverse scaling
unscaled_out_data = scaler.inverse_transform(out_data)
df_unscaled = pd.DataFrame(unscaled_out_data, columns=df_scaled.columns, index=out_data.index) # use columns from df_scaled

# Restore removed columns
df_with_removed_columns = df_unscaled.join(removed_columns_df.loc[df_unscaled.index])

In [34]:
print("Data after inverse scaling and adding removed columns:")
print(df_with_removed_columns.head())
print(df_with_removed_columns.info())

Data after inverse scaling and adding removed columns:
   space_group_core  space_group_shell     Tc     x     y     z  shape  \
1             225.0              227.0  850.0   7.0   7.0   7.0    1.0   
2             227.0                0.0  397.0  17.0  17.0  17.0    1.0   
3             227.0                0.0  397.0  11.0  11.0  11.0    1.0   
4             227.0                0.0  397.0  12.0  12.0  12.0    1.0   
5             227.0                0.0  397.0  18.0  18.0  18.0    1.0   

   temperature_k  h_range_max_koe  sat_em_g  ...  shell_alpha  shell_beta  \
1            5.0             50.0     83.14  ...    119.97187  119.806604   
2          300.0             10.0      2.29  ...      0.00000    0.000000   
3          300.0             10.0      2.43  ...      0.00000    0.000000   
4          300.0             10.0     16.15  ...      0.00000    0.000000   
5          300.0             10.0     18.75  ...      0.00000    0.000000   

   shell_gamma        Aex  core_c/a  

In [35]:
df = df_with_removed_columns
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 1899 entries, 1 to 2142
Data columns (total 31 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   space_group_core   1899 non-null   float64
 1   space_group_shell  1899 non-null   float64
 2   Tc                 1899 non-null   float64
 3   x                  1899 non-null   float64
 4   y                  1899 non-null   float64
 5   z                  1899 non-null   float64
 6   shape              1899 non-null   float64
 7   temperature_k      1899 non-null   float64
 8   h_range_max_koe    1899 non-null   float64
 9   sat_em_g           1899 non-null   float64
 10  coer_oe            1899 non-null   float64
 11  mr (emu/g)         1899 non-null   float64
 12  sphericity         1899 non-null   float64
 13  max/min            1899 non-null   float64
 14  area/volume        1899 non-null   float64
 15  num_of_magn_ions   1899 non-null   float64
 16  core_b             1899 non-n

In [36]:
# Export data
df.to_csv(r'Megy_without_outliners.csv')